In [1]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
import numpy as np

In [2]:
# Load CSV file
csv_file = "networks_assignment.csv"
df = pd.read_csv(csv_file)


In [3]:
# Extract node labels
node_labels = df.columns[1:].tolist()  # Column headers as node names
source_nodes = df.iloc[:, 0].tolist()  # First column as source nodes

In [4]:
# Create a directed graph
G = nx.DiGraph()

# Add edges with weights
for i, src in enumerate(source_nodes):
    for j, dest in enumerate(node_labels):
        weight = df.iloc[i, j + 1]  # Get weight from adjacency matrix
        if weight > 0:  # Only add edges with nonzero weight
            G.add_edge(src, dest, weight=weight)

In [5]:
# Define node groups and colors
blue_nodes = {'N', 'D', 'F', 'I', 'S'}
green_nodes = {'BIH', 'GEO', 'ISR', 'MNE', 'SRB', 'CHE', 'TUR', 'UKR', 'GBR', 'AUS', 'HKG', 'USA'}
yellow_nodes = {'AUT', 'BEL', 'BGR', 'HRV', 'CZE', 'EST', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LUX', 'NLD', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP'}
node_colors = {}

for node in G.nodes():
    if node in blue_nodes:
        node_colors[node] = 'blue'
    elif node in green_nodes:
        node_colors[node] = 'green'
    elif node in yellow_nodes:
        node_colors[node] = 'yellow'
    else:
        node_colors[node] = 'red'  # Default color

In [6]:
# Define a pentagon layout for (D, F, I, N, S)
center_nodes = ["D", "F", "I", "N", "S"]
NODES_NUM = len(center_nodes)
RADIUS = 0.8

# Compute pentagon positions
pentagon_pos = {
    center_nodes[i]: (
        RADIUS * np.cos(2 * np.pi * i / NODES_NUM), 
        RADIUS * np.sin(2 * np.pi * i / NODES_NUM)
    ) 
    for i in range(NODES_NUM)
}

In [7]:
# Compute positions for remaining nodes outside the pentagon
remaining_nodes = set(G.nodes()) - set(center_nodes)
pos = nx.spring_layout(G, seed=42)  # Compute positions for all nodes

# Adjust positions of external nodes to be farther from center
SCALE_FACTOR = 5
for node in remaining_nodes:
    pos[node] = (pos[node][0] * SCALE_FACTOR, pos[node][1] * SCALE_FACTOR)

# Override pentagon node positions
pos.update(pentagon_pos)

In [8]:
# Create Plotly traces
edge_x, edge_y, edge_weights = [], [], []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])  # None creates separate lines for each edge
    edge_y.extend([y0, y1, None])
    edge_weights.append(edge[2]["weight"])

# Create the edge trace
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color="#aaa"),
    hoverinfo="text",
    text=[f"Weight: {w}" for w in edge_weights],
    mode="lines"
)

# Create the node trace
node_x, node_y, node_color, node_text = [], [], [], []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_color.append(node_colors.get(node, 'red'))  # Default to red if missing
    node_text.append(node)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode="markers+text",
    marker=dict(size=32, color=node_color),
    text=node_text,
    textfont=dict(size=12, color='black', family='Arial'),
    textposition="middle center",
    hoverinfo="text"
)

In [9]:
# Create the final figure
fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    title="Network Graph with Pentagram",
    showlegend=False,
    hovermode="closest",
    margin=dict(b=10, l=10, r=10, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# Show the figure
fig.show()